In [34]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import onnx
import keras2onnx
from numpy import random
import onnxruntime
import time

In [27]:
input_shape = (15,15,2)
model_name_onnx = "onnx_inference_model.onnx"

def create_model():
    return keras.Sequential(
        [
            keras.Input(shape=input_shape, name="input"),
            layers.Conv2D(16, kernel_size=(5, 5), activation="relu"),
            layers.Conv2D(8, kernel_size=(3, 3), activation="relu"),
            layers.Flatten(),
            #layers.Dropout(0.5),
            layers.Dense(2, activation="relu"),
            layers.Dense(1, activation="sigmoid", name="output")
        ]
    )
model = create_model()

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 11, 11, 16)        816       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 9, 9, 8)           1160      
_________________________________________________________________
flatten (Flatten)            (None, 648)               0         
_________________________________________________________________
dense (Dense)                (None, 2)                 1298      
_________________________________________________________________
output (Dense)               (None, 1)                 3         
Total params: 3,277
Trainable params: 3,277
Non-trainable params: 0
_________________________________________________________________


In [28]:
onnx_model = keras2onnx.convert_keras(model, model.name)

onnx.save_model(onnx_model, model_name_onnx)

tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 19 -> 13
The maximum opset needed by this model is only 11.


In [29]:
session = onnxruntime.InferenceSession(model_name_onnx)  

In [36]:
data = random.rand(10000,input_shape[0],input_shape[1],input_shape[2]).astype(np.single)
print(data)
print(type(data[0]))

[[[[0.82053363 0.9608356 ]
   [0.06297879 0.38979977]
   [0.09433876 0.27077416]
   ...
   [0.14184965 0.5104971 ]
   [0.57435036 0.593198  ]
   [0.01278451 0.02785275]]

  [[0.21489012 0.537082  ]
   [0.89448035 0.2835876 ]
   [0.51925784 0.2712296 ]
   ...
   [0.1923579  0.49554566]
   [0.02352868 0.04423856]
   [0.74857223 0.7702438 ]]

  [[0.64332545 0.86404943]
   [0.19336165 0.8971832 ]
   [0.27870715 0.88027805]
   ...
   [0.42786178 0.597311  ]
   [0.42772645 0.19293176]
   [0.9809513  0.4359652 ]]

  ...

  [[0.7908211  0.7975447 ]
   [0.9569479  0.8067809 ]
   [0.95295537 0.73899   ]
   ...
   [0.7408552  0.65267926]
   [0.81699204 0.68511325]
   [0.6640459  0.66994196]]

  [[0.00835647 0.7393267 ]
   [0.99204427 0.37042826]
   [0.9554394  0.4849424 ]
   ...
   [0.9849971  0.5137958 ]
   [0.481561   0.75081235]
   [0.1843084  0.05024053]]

  [[0.01669408 0.86375016]
   [0.5922881  0.56586456]
   [0.815541   0.9427222 ]
   ...
   [0.4906323  0.79118454]
   [0.47980627 0.725099

In [38]:
print("starting inference")
start = time.time()
for i in range(1000):
    prediction = session.run(None, {"input": data} )
duration = time.time() - start
print(f"inference ended in {duration} seconds")

starting inference
inference ended in 9.35300064086914 seconds
